In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
data = pd.read_csv('dataset/mental_health_dataset.csv') # Gets data from model
print(data.head())

   User_ID  Age      Gender  Occupation    Country Mental_Health_Condition  \
0        1   36  Non-binary       Sales     Canada                      No   
1        2   34      Female   Education         UK                     Yes   
2        3   65  Non-binary       Sales        USA                     Yes   
3        4   34        Male       Other  Australia                      No   
4        5   22      Female  Healthcare     Canada                     Yes   

  Severity Consultation_History Stress_Level  Sleep_Hours  Work_Hours  \
0   Medium                  Yes       Medium          7.1          46   
1      NaN                   No          Low          7.5          47   
2     High                   No          Low          8.4          58   
3      Low                   No       Medium          9.8          30   
4      Low                   No       Medium          4.9          62   

   Physical_Activity_Hours  
0                        5  
1                        8  
2    

In [ ]:
pd.get_dummies(data)

In [ ]:
# getting input for variables (eventually to be replaced by web-app
# usr_age = input('Enter your age: ')
# usr_gender = input('Enter your gender (M, F, NB, NA): ')
# usr_sleep = input('How many hours a night do you sleep? : ')
# usr_work = input('How many hours a week do you work? : ')
# usr_fitness = input('How many hours a week do you excercise? : ')
# usr_consult = input('Have you been consulted for a mental health condition in the past?: ')
# usr_stress = input('What is your stress level?: ')


In [ ]:
#correlation btwn age and hours worked
plt.figure(figsize=(10, 6))
plt.scatter(data['Age'], data['Work_Hours'], alpha=0.5)
plt.title("Correlation between Age and Work Hours")
plt.xlabel("Age")
plt.ylabel("Work Hours")
plt.grid(True)
plt.show()

In [ ]:
plt.bar(data['Gender'], data['Sleep_Hours'], color='blue')

In [4]:
label_encoders = {}
categorical_columns = ['Gender', 'Occupation', 'Country', 'Severity', 'Consultation_History', 'Stress_Level']

In [5]:
# loop to convert values into numerical data that can be used by the model
for column in categorical_columns: 
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

In [6]:
# selects target data, which is mental health condition in this case (we are trying to predict it)
target_encoder = LabelEncoder()
data['Mental_Health_Condition'] = target_encoder.fit_transform(data['Mental_Health_Condition'])

In [7]:
# sets features and target
features = ['Age', 'Gender', 'Stress_Level', 'Sleep_Hours', 'Work_Hours', 'Physical_Activity_Hours']
target = 'Mental_Health_Condition'

In [9]:
X = data[features] # conventional names to represent feats and target are X and y
y = data[target]

In [10]:
#for normalizing data (scales x so mean to 0 and SD to 1)
scaler = StandardScaler()
X = scaler.fit_transform(X) 